In [1]:
import os
import pandas as pd
import seaborn as sns
import random
from datetime import date
from dateutil.relativedelta import relativedelta

from sklearn.metrics import mean_squared_error, confusion_matrix

In [2]:
data_folder = '../03.clean-data/'
results_folder = '../04.results/'
positive_monthly_return_cutoff = 0.002


In [3]:
if not os.path.isdir(results_folder):
    if not os.path.exists(results_folder):
        os.mkdir(results_folder)
    else:
        raise NotADirectoryError

## Load Data

In [4]:
hpi = pd.read_csv(os.path.join(data_folder, 'hpi.csv'), parse_dates=['Date']) # Load
hpi.sort_values(['Area', 'Type', 'Date'], inplace=True) # Sort
hpi.reset_index(inplace=True)

## Calculate Monthly Changes ("Returns")

In [5]:
hpi_returns = hpi.copy()

In [6]:
hpi_returns['HPI.L1'] = hpi_returns.groupby(['Area', 'Type'])['HPI'].shift(1)
hpi_returns['HPI.L12'] = hpi_returns.groupby(['Area', 'Type'])['HPI'].shift(12)
hpi_returns['HPI.F2'] = hpi_returns.groupby(['Area', 'Type'])['HPI'].shift(-2)
hpi_returns['HPI.F5'] = hpi_returns.groupby(['Area', 'Type'])['HPI'].shift(-5)

In [7]:
hpi_returns['MonthlyChangeInHPI'] = (hpi_returns['HPI'] - hpi_returns['HPI.L1']) / hpi_returns['HPI.L1']
hpi_returns['YearlyChangeInHPI'] = (hpi_returns['HPI'] - hpi_returns['HPI.L12']) / hpi_returns['HPI.L12']
hpi_returns['FutureQuarterlyChangeInHPI'] = (hpi_returns['HPI.F2'] - hpi_returns['HPI.L1']) / hpi_returns['HPI.L1']
hpi_returns['FutureSemiAnnualChangeInHPI'] = (hpi_returns['HPI.F5'] - hpi_returns['HPI.L1']) / hpi_returns['HPI.L1']
hpi_returns['PositiveMonthlyReturn'] = (hpi_returns['MonthlyChangeInHPI'] > positive_monthly_return_cutoff)


## Use the Rolling Average to Make a Forecast

In [8]:
window_length = 36  # months before the date we run the forecast on.


### Example with one date

In [9]:
start_date = pd.to_datetime(date(2012, 2, 1))
end_date = start_date + relativedelta(months=window_length)
train = hpi_returns[hpi_returns['Date'].between(start_date, end_date)].copy()
date_of_forecast_being_made = train['Date'].max()
date_of_forecasted_value = end_date + relativedelta(months=1)
start_date, end_date, date_of_forecast_being_made, date_of_forecasted_value

(Timestamp('2012-02-01 00:00:00'),
 Timestamp('2015-02-01 00:00:00'),
 Timestamp('2015-02-01 00:00:00'),
 Timestamp('2015-03-01 00:00:00'))

In [10]:
forecasted_monthly_return = train.groupby(['Area', 'Type'])['MonthlyChangeInHPI'].mean()
forecasted_monthly_return.rename('ForecastedMonthlyChangeInHPI', inplace=True)
forecasted_monthly_return = forecasted_monthly_return.to_frame()

forecasted_monthly_return['ForecastedPositiveReturn'] = (
    train
        [train['MonthlyChangeInHPI'].notnull()]
        .groupby(['Area', 'Type'])
        ['MonthlyChangeInHPI']
        .apply(lambda x: (x > positive_monthly_return_cutoff).sum() / len(x))
)

forecasted_monthly_return['Date'] = date_of_forecasted_value
forecasted_monthly_return.reset_index(inplace=True)
forecasted_monthly_return

,Area,Type,ForecastedMonthlyChangeInHPI,ForecastedPositiveReturn,Date
0,TREB Total,Apartment,0.003106,0.482759,2015-03-01
1,TREB Total,Composite,0.006060,0.620690,2015-03-01
2,TREB Total,Single-Family Attached,0.006839,0.655172,2015-03-01
3,TREB Total,Single-Family Detached,0.006819,0.655172,2015-03-01
4,TREB Total,Townhouse,0.005879,0.724138,2015-03-01
...,...,...,...,...,...
175,Toronto W10,Apartment,0.003104,0.517241,2015-03-01
176,Toronto W10,Composite,0.007192,0.655172,2015-03-01
177,Toronto W10,Single-Family Attached,0.007635,0.655172,2015-03-01
178,Toronto W10,Single-Family Detached,0.008944,0.724138,2015-03-01


### Rolling predictions on a loop

In [11]:
possible_start_dates = pd.date_range(
    start=hpi_returns['Date'].min(),
    end=hpi_returns['Date'].max() - relativedelta(months=window_length - 1),
    freq='MS',
)
len(possible_start_dates)

76

In [12]:
%%time
forecast_dfs = []
for start_date in possible_start_dates:
    end_date = start_date + relativedelta(months=window_length)
    train = hpi_returns[hpi_returns['Date'].between(start_date, end_date)].copy()
    date_of_forecast_being_made = train['Date'].max()
    date_of_forecasted_value = end_date + relativedelta(months=1)
    
    forecast_df = train.groupby(['Area', 'Type'])['MonthlyChangeInHPI'].mean()
    forecast_df.rename('ForecastedMonthlyChangeInHPI', inplace=True)
    forecast_df = forecast_df.to_frame() 
    forecast_df['ForecastedQuarterlyChangeInHPI'] = (1 + forecast_df['ForecastedMonthlyChangeInHPI']) ** 3 - 1
    forecast_df['ForecastedSemiAnnualChangeInHPI'] = (1 + forecast_df['ForecastedMonthlyChangeInHPI']) ** 6 - 1
    
    forecast_df['ForecastedPositiveReturn'] = (
        train
            [train['MonthlyChangeInHPI'].notnull()]
            .groupby(['Area', 'Type'])
            ['MonthlyChangeInHPI']
            .apply(lambda x: (x > positive_monthly_return_cutoff).sum() / len(x))
        )
    
    forecast_df['Date'] = date_of_forecasted_value
    forecast_df.reset_index(inplace=True)
    
    forecast_dfs.append(forecast_df)

forecasted_monthly_return = pd.concat(forecast_dfs)
forecasted_monthly_return

Wall time: 4.24 s


,Area,Type,ForecastedMonthlyChangeInHPI,ForecastedQuarterlyChangeInHPI,ForecastedSemiAnnualChangeInHPI,ForecastedPositiveReturn,Date
0,TREB Total,Apartment,0.003106,0.009348,0.018784,0.482759,2015-03-01
1,TREB Total,Composite,0.006060,0.018290,0.036915,0.620690,2015-03-01
2,TREB Total,Single-Family Attached,0.006839,0.020658,0.041742,0.655172,2015-03-01
3,TREB Total,Single-Family Detached,0.006819,0.020597,0.041618,0.655172,2015-03-01
4,TREB Total,Townhouse,0.005879,0.017742,0.035799,0.724138,2015-03-01
...,...,...,...,...,...,...,...
300,Whitchurch-Stouffville,Apartment,0.001982,0.005956,0.011948,0.485714,2021-06-01
301,Whitchurch-Stouffville,Composite,0.009300,0.028159,0.057112,0.685714,2021-06-01
302,Whitchurch-Stouffville,Single-Family Attached,0.013590,0.041328,0.084364,0.657143,2021-06-01
303,Whitchurch-Stouffville,Single-Family Detached,0.009514,0.028813,0.058456,0.628571,2021-06-01


## Validate

In [13]:
validation = hpi_returns.merge(
    forecasted_monthly_return,
    how='outer',
    on=['Area', 'Type', 'Date'],
)
print(len(validation), len(validation.drop_duplicates(['Area', 'Type', 'Date'])))  # Check if the merge (join) impacted the granularity of the data set, i.e. if we introduced duplicates
validation = validation[validation['ForecastedMonthlyChangeInHPI'].notnull() & validation['MonthlyChangeInHPI'].notnull()]

28080 28080


In [14]:
validation.drop(columns=['level_0', 'index']).to_csv(os.path.join(results_folder, '05.a.benchmark_forecasts.csv'), index=False)

### MSE

In [15]:
benchmark_monthly_mse = mean_squared_error(validation['MonthlyChangeInHPI'], validation['ForecastedMonthlyChangeInHPI'])
benchmark_monthly_mse

0.0008836380148939758

In [16]:
quarterly_change_validation = validation[validation['FutureQuarterlyChangeInHPI'].notnull() & validation['ForecastedQuarterlyChangeInHPI'].notnull()]
benchmark_quarterly_mse = mean_squared_error(quarterly_change_validation['FutureQuarterlyChangeInHPI'], quarterly_change_validation['ForecastedQuarterlyChangeInHPI'])
benchmark_quarterly_mse

0.003612522174281026

In [17]:
semi_annual_change_validation = validation[validation['FutureSemiAnnualChangeInHPI'].notnull() & validation['ForecastedSemiAnnualChangeInHPI'].notnull()]
benchmark_semi_annual_mse = mean_squared_error(semi_annual_change_validation['FutureSemiAnnualChangeInHPI'], semi_annual_change_validation['ForecastedSemiAnnualChangeInHPI'])
benchmark_semi_annual_mse

0.008520661692762093

### City of Toronto MSE

In [18]:
validation_for_city_of_toronto = validation[validation['Area'].str.startswith('Toronto')]

benchmark_city_of_toronto_mse = mean_squared_error(
    validation_for_city_of_toronto['MonthlyChangeInHPI'], 
    validation_for_city_of_toronto['ForecastedMonthlyChangeInHPI'])
benchmark_city_of_toronto_mse

0.0010167614397122388

### Normalize the MSE

In [19]:
# MSE is 10% of the average value.
benchmark_mse / hpi_returns['MonthlyChangeInHPI'].mean()

NameError: name 'benchmark_mse' is not defined

In [ ]:
benchmark_mse / hpi_returns.query('Area == "TREB Total" & Type == "Composite"')['MonthlyChangeInHPI'].mean()

### MSE by Area & Housing Type

In [ ]:
benchmark_mse_by_area_and_type = (
    validation
        .groupby(['Area', 'Type'])
        .apply(lambda x: mean_squared_error(x['MonthlyChangeInHPI'], x['ForecastedMonthlyChangeInHPI']))
)
benchmark_mse_by_area_and_type

### MSE by Year

In [ ]:
benchmark_mse_by_year = (
    validation
        .groupby(validation['Date'].dt.year)
        .apply(lambda x: mean_squared_error(x['MonthlyChangeInHPI'], x['ForecastedMonthlyChangeInHPI']))
)
benchmark_mse_by_year

### City of Toronto MSE by Year

In [ ]:
validation_for_city_of_toronto = validation[validation['Area'].str.startswith('Toronto')]

benchmark_city_of_toronto_mse_by_year = (
    validation
        .groupby(validation_for_city_of_toronto['Date'].dt.year)
        .apply(lambda x: mean_squared_error(x['MonthlyChangeInHPI'], x['ForecastedMonthlyChangeInHPI']))
)
benchmark_city_of_toronto_mse_by_year

## Likelihood of a Positive Return

In [20]:
assert hpi_returns['PositiveMonthlyReturn'].notnull().all()

hpi_returns['PositiveMonthlyReturn'].sum() / len(hpi_returns['PositiveMonthlyReturn'])

0.5706479601255308

In [21]:
confusion_matrix(hpi_returns['PositiveMonthlyReturn'].astype(int), [1] * len(hpi_returns), normalize='pred')

array([[0.        , 0.42935204],
       [0.        , 0.57064796]])

### Likelihood of a Positive Return by Area and Housing Type

In [22]:
likelihood = (
    hpi_returns.groupby(['Area', 'Type'])['MonthlyChangeInHPI'].apply(lambda x: (x > positive_monthly_return_cutoff).sum())
        /
    hpi_returns[hpi_returns['MonthlyChangeInHPI'].notnull()].groupby(['Area', 'Type']).apply(len)
)
likelihood.rename('PositiveReturnLikelihood', inplace=True)

Area                    Type                  
Adjala-Tosorontio       Apartment                      NaN
                        Composite                 0.705882
                        Single-Family Attached         NaN
                        Single-Family Detached    0.705882
                        Townhouse                      NaN
                                                    ...   
Whitchurch-Stouffville  Apartment                 0.444444
                        Composite                 0.661765
                        Single-Family Attached    0.647059
                        Single-Family Detached    0.647059
                        Townhouse                 0.488889
Name: PositiveReturnLikelihood, Length: 305, dtype: float64

In [23]:
likelihood.reset_index().to_csv(os.path.join(results_folder, '05.a.positive_return_likelihood.csv'), index=False)